In [20]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import shutil
from torchvision.io import read_image
import pandas as pd

In [27]:
#copy non-plural material into a new folder
target_folder = "material_im_non_plu/"
source_folder = "material_images/"
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

list = os.listdir(source_folder)
for file_header in list:
    count = 0
    for char in file_header:
        if char == '1':
            count += 1
    if count == 1:
        shutil.copytree(source_folder + file_header, target_folder + file_header)
